In [314]:
import pandas as pd
#import seaborn as sns

import plotly.graph_objects as go
#from plotly.subplots import make_subplots

from scipy import stats
from datetime import datetime

Obtaining the datasets and adding the average row for the gas dataset

In [315]:
meralco_df = pd.read_csv('MonthlyPumpPrices.csv', parse_dates=['Date'])
ron95_data = meralco_df[meralco_df['Product'] == "Gasoline (RON95)"].sort_values('Date')
ron95_data = ron95_data[(ron95_data['Date'] >= '2020-01-01') & 
                       (ron95_data['Date'] <= '2025-01-01')]
ron95_data['Average Price'] = (ron95_data['Overall Range Min'] + ron95_data['Overall Range Max']) / 2


Monthly gas prices graph: (from 2020 to 2025)

In [316]:
# Color definitions
dry_season_color = 'rgba(255, 165, 0, 0.2)'  # Orange
wet_season_color = 'rgba(0, 191, 255, 0.2)'   # Light Blue
price_line_color = '#DC143C'                   # Crimson
price_range_color = 'rgba(158, 27, 50, 0.5)'  # Dark Red with transparency

# Create figure
fig = go.Figure()

# Add seasonal background shapes first (so they appear behind the data)
for year in range(2019, 2025):
    # Dry season (Dec-May)
    fig.add_vrect(
        x0=max(datetime(year, 12, 1), datetime(2020, 1, 1)),
        x1=min(datetime(year+1, 6, 1), datetime(2024, 12, 31)),
        fillcolor=dry_season_color,
        layer="below",
        line_width=0
    )
    
    # Wet season (Jun-Nov)
    fig.add_vrect(
        x0=max(datetime(year, 6, 1), datetime(2020, 1, 1)),
        x1=min(datetime(year, 12, 1), datetime(2024, 12, 31)),
        fillcolor=wet_season_color,
        layer="below",
        line_width=0
    )

# Add price range (fill between)
fig.add_trace(go.Scatter(
    x=ron95_data['Date'],
    y=ron95_data['Overall Range Max'],
    fill=None,
    mode='lines',
    line=dict(width=0),
    showlegend=False,
    hoverinfo='skip'
))

fig.add_trace(go.Scatter(
    x=ron95_data['Date'],
    y=ron95_data['Overall Range Min'],
    fill='tonexty',
    mode='lines',
    fillcolor=price_range_color,
    line=dict(width=0),
    name='Price Range',
    hovertemplate="<b>%{x|%b %Y}</b><br>Range: %{y:.2f}–%{text} pesos<extra></extra>",
    text=ron95_data['Overall Range Max'].round(2)
))

# Add average price line
fig.add_trace(go.Scatter(
    x=ron95_data['Date'],
    y=ron95_data['Average Price'],
    mode='lines+markers',
    line=dict(color=price_line_color, width=3),
    marker=dict(size=8),
    name='Average Price',
    hovertemplate="<b>%{x|%b %Y}</b><br>Avg: %{y:.2f} pesos<extra></extra>"
))

# Layout customization
fig.update_layout(
    title={
        'text': "<b>Cost per Liter for Gasoline (RON95) Monthly Price Trends (2020-2024)</b>",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'font': dict(size=18, color='#333333')
    },
    xaxis=dict(
        title="Date",
        range=['2020-01-01', '2024-12-31'],
        tickformat="%Y",
        dtick="M12",  # Yearly ticks
        showgrid=False
    ),
    yaxis=dict(
        title="Cost (pesos)",
        showgrid=True,
        gridcolor='rgba(0,0,0,0.1)'
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    plot_bgcolor='#FAFAFA',
    paper_bgcolor='white',
    hovermode="x unified",
    margin=dict(t=100)
)

# Custom legend for seasons (requires adding dummy traces)
fig.add_trace(go.Scatter(
    x=[None],
    y=[None],
    mode='markers',
    marker=dict(size=15, color=dry_season_color),
    name='Dry Season (Dec-May)'
))

fig.add_trace(go.Scatter(
    x=[None],
    y=[None],
    mode='markers',
    marker=dict(size=15, color=wet_season_color),
    name='Wet Season (Jun-Nov)'
))

fig.show()

# Save as HTML
#fig.write_html("gasoline_price_trends_interactive.html")

Monthly electricity prices graph (2020 to 2025)

In [317]:
## Relevant functions


def get_specific_kwh_df(kwh_consumption: int):
    meralco_df = pd.read_csv('MeralcoSummaryOfRates.csv')
    meralco_df['Date'] = pd.to_datetime(meralco_df['Year'].astype(str) + '-' + meralco_df['Month'].astype(str), format='%Y-%m')

    # Reorder the columns to make 'Date' the first column
    cols = ['Date'] + [col for col in meralco_df.columns if col != 'Date']
    meralco_df = meralco_df[cols]

    chosen_kwh_data = meralco_df[(meralco_df['kWh Consumption'] == kwh_consumption) & (meralco_df['For Lifeline Customers'] == False)]
    chosen_kwh_data = chosen_kwh_data[(chosen_kwh_data['Date'].dt.year >= 2020) & (chosen_kwh_data['Date'].dt.year <= 2024)]

    return chosen_kwh_data

def plot_specific_kwh_df(kwh_consumption: int = 200) -> None:
    df_to_plot = get_specific_kwh_df(kwh_consumption)

    # Color definitions
    dry_season_color = 'rgba(204, 102, 0, 0.3)'  # #CC6600 with alpha
    wet_season_color = 'rgba(102, 153, 102, 0.3)' # #669966 with alpha
    line_color = 'rgba(0, 0, 255, 1)'            # Blue

    # Create figure
    fig = go.Figure()

    # Add seasonal background shapes first (so they appear behind the data)
    for year in range(2019, 2025):
        # Dry season (Dec-May)
        fig.add_vrect(
            x0=max(datetime(year, 12, 1), datetime(2020, 1, 1)),
            x1=min(datetime(year+1, 6, 1), datetime(2024, 12, 31)),
            fillcolor=dry_season_color,
            layer="below",
            line_width=0,
        )
        
        # Wet season (Jun-Nov)
        fig.add_vrect(
            x0=max(datetime(year, 6, 1), datetime(2020, 1, 1)),
            x1=min(datetime(year, 12, 1), datetime(2024, 12, 31)),
            fillcolor=wet_season_color,
            layer="below",
            line_width=0,
        )

    # Add main line plot
    fig.add_trace(go.Scatter(
        x=df_to_plot['Date'],
        y=df_to_plot['Total Bill'],
        mode='lines+markers',
        line=dict(color=line_color, width=2),
        marker=dict(size=8),
        name='Electricity Cost',
        hovertemplate="<b>%{x|%b %Y}</b><br>Cost: %{y:.2f} pesos<extra></extra>"
    ))

    # Layout customization
    fig.update_layout(
        title={
            'text': f"<b>Meralco Cost per kW for {kwh_consumption} kWh Consumption (2020-2024, Non-Lifeline)</b>",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'font': dict(size=18, color='#333333')
        },
        xaxis=dict(
            title="Date",
            range=['2020-01-01', '2024-12-31'],
            tickformat="%Y",
            dtick="M12",  # Yearly ticks
            showgrid=False
        ),
        yaxis=dict(
            title="Cost (pesos)",
            showgrid=True,
            gridcolor='rgba(0,0,0,0.1)'
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        plot_bgcolor='#FAFAFA',
        paper_bgcolor='white',
        hovermode="x unified"
    )
    fig.add_trace(go.Scatter(
    x=[None],
    y=[None],
    mode='markers',
    marker=dict(size=15, color=dry_season_color),
    name='Dry Season (Dec-May)'
    ))

    fig.add_trace(go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        marker=dict(size=15, color=wet_season_color),
        name='Wet Season (Jun-Nov)'
    ))

    fig.show()

    # Optional: Save as HTML
    # fig.write_html(f"electricity_cost_{kwh_consumption}kWh.html")


In [318]:
plot_specific_kwh_df(200)

Seasonality for raw gas and electricity (Just curious)

In [319]:
res = []
chosen_kwh_data = get_specific_kwh_df(200)
for i in chosen_kwh_data['Year'].unique():
    res.append(chosen_kwh_data[chosen_kwh_data['Year'] == i]['Total Bill'].values)

print(f"Electricity, Yearly seasonality: {stats.kruskal(*res)}")

res = []
for i in ron95_data['Date'].dt.year.unique():
    year = ron95_data['Date'].dt.year
    res.append(ron95_data[year == i]['Average Price'].values)

print(f"Gas, Yearly seasonality: {stats.kruskal(*res)}")

res = []
for i in chosen_kwh_data['Month'].unique():
    res.append(chosen_kwh_data[chosen_kwh_data['Month'] == i]['Total Bill'].values)

print(f"Electricity, Monthly seasonality: {stats.kruskal(*res)}")

res = []
for i in ron95_data['Date'].dt.month.unique():
    month = ron95_data['Date'].dt.month
    res.append(ron95_data[month == i]['Average Price'].values)

print(f"Gas, Monthly seasonality: {stats.kruskal(*res)}")

Electricity, Yearly seasonality: KruskalResult(statistic=48.66885245901639, pvalue=6.845482130987131e-10)
Gas, Yearly seasonality: KruskalResult(statistic=46.01031200423054, pvalue=9.038308313091285e-09)
Electricity, Monthly seasonality: KruskalResult(statistic=1.1455737704918079, pvalue=0.9998998777588135)
Gas, Monthly seasonality: KruskalResult(statistic=1.5203595980962632, pvalue=0.9995937999261401)


Comparing gas and electricity

In [320]:
def make_comparison_dataframe(km_l: float, km_kwh: float, kwh_consumption: int = 200):
    df_gas = ron95_data.copy()
    df_elec = get_specific_kwh_df(kwh_consumption)

    #print(df_elec)

    # Drop excess month in df_gas
    date_to_drop = pd.to_datetime('2025-01-01')
    df_gas = df_gas[df_gas['Date'] != date_to_drop]

    monthly_date_range = pd.date_range(start='2020-01-01', end='2024-12-31', freq='MS')
    # Create a new DataFrame with the monthly date range
    df_output = pd.DataFrame({'Date': monthly_date_range.date,
                              'Gas cost/km':None,
                              'Electricity cost/km':None,
                              'Difference': None})
    
    df_gas['Date'] = pd.to_datetime(df_gas['Date'])
    df_elec['Date'] = pd.to_datetime(df_elec['Date'])
    df_output['Date'] = pd.to_datetime(df_output['Date'])

    df_gas = df_gas.set_index('Date')
    df_elec = df_elec.set_index('Date')
    df_output = df_output.set_index('Date')

    # Calculate cost per km for each row
    for date in df_output.index:
        gas_price = df_gas.loc[date, 'Average Price']
        df_output.loc[date, 'Gas cost/km'] = gas_price / km_l  # PHP per km

        elec_price = df_elec.loc[date, 'Total Bill']
        df_output.loc[date, 'Electricity cost/km'] = elec_price / km_kwh  # PHP per km

    # Calculate the difference (Gas cost - Electricity cost)
    df_output['Difference'] = df_output['Gas cost/km'] - df_output['Electricity cost/km']
    df_output = df_output.reset_index()
    return df_output

# toyota vios (15) VS byd ATTO 3 (6.25)
"""
test data
30, 40, 50
6.25, 7, 7.5
"""

'\ntest data\n30, 40, 50\n6.25, 7, 7.5\n'

Plotting the dataframe comparison

In [321]:
def plot_comparison_dataframe(km_l: float, km_kwh: float, kwh_consumption: int = 200):
    df = make_comparison_dataframe(km_l, km_kwh, kwh_consumption)

    # Color definitions
    dry_season_color = 'rgba(204, 102, 0, 0.3)'  # #CC6600 with alpha
    wet_season_color = 'rgba(102, 153, 102, 0.3)' # #669966 with alpha
    gas_color = '#EF553B'  # Red
    electricity_color = '#636EFA'  # Blue

    fig = go.Figure()

    ## vertical lines
    luzon_lockdown = datetime(2020, 3, 1)
    lockdown_reduced = datetime(2021, 10, 1)
    ukraine_invasion = datetime(2022, 2, 1)
    fig.update_layout(
        shapes=[
            dict(
                type="line",
                x0=luzon_lockdown,
                x1=luzon_lockdown,
                line=dict(color="black", width=2, dash="dash")
            ),
            dict(
                type="line",
                x0=lockdown_reduced,
                x1=lockdown_reduced,
                line=dict(color="black", width=2, dash="dash")
            ),
            dict(
                type="line",
                x0=ukraine_invasion,
                x1=ukraine_invasion,
                line=dict(color="black", width=2, dash="dash")
            ),
        ],
        annotations=[
            dict(
                x=luzon_lockdown,
                xref="x",
                showarrow=False,
                text="Luzon ECQ Start",
                textangle=-90,
                xanchor="right",
                yanchor="bottom",
                font=dict(size=10),
            ),
            dict(
                x=lockdown_reduced,
                xref="x",
                showarrow=False,
                text="ECQ to MECQ",
                textangle=-90,
                xanchor="right",
                yanchor="bottom",
                font=dict(size=10),
            ),
            dict(
                x=ukraine_invasion,
                xref="x",
                showarrow=False,
                text="Rus invades Ukr",
                textangle=-90,
                xanchor="right",
                yanchor="bottom",
                font=dict(size=10),
            )
        ],
    )

    # Add seasonal background shapes first (so they appear behind the data)
    for year in range(2019, 2025):
        # Dry season (Dec-May)
        fig.add_vrect(
            x0=max(datetime(year, 12, 1), df['Date'].min()),
            x1=min(datetime(year+1, 6, 1), df['Date'].max()),
            fillcolor=dry_season_color,
            layer="below",
            line_width=0,
        )
        
        # Wet season (Jun-Nov)
        fig.add_vrect(
            x0=max(datetime(year, 6, 1), df['Date'].min()),
            x1=min(datetime(year, 12, 1), df['Date'].max()),
            fillcolor=wet_season_color,
            layer="below",
            line_width=0,
        )

    # Add Gas line plot
    fig.add_trace(go.Scatter(
        x=df['Date'],
        y=df['Gas cost/km'],
        mode='lines+markers',
        line=dict(color=gas_color, width=2),
        marker=dict(size=8),
        name='Gas Cost/km',
        hovertemplate="<b>%{x|%b %Y}</b><br>Gas Cost: PHP %{y:.3f}<extra></extra>"
    ))

    # Add Electricity line plot
    fig.add_trace(go.Scatter(
        x=df['Date'],
        y=df['Electricity cost/km'],
        mode='lines+markers',
        line=dict(color=electricity_color, width=2),
        marker=dict(size=8),
        name='Electricity Cost/km',
        hovertemplate="<b>%{x|%b %Y}</b><br>Electricity Cost: PHP %{y:.3f}<extra></extra>"
    ))

    fig.update_layout(
        title={
            'text': "<b>Gas vs. Electricity Cost per Kilometer (2020-2025)</b>",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'font': dict(size=18, color='#333333')
        },
        xaxis=dict(
            title="Date",
            range=[df['Date'].min(), df['Date'].max()],
            tickformat="%Y",
            dtick="M12",  # Yearly ticks
            showgrid=False
        ),
        yaxis=dict(
            title="Cost per km (PHP)",
            showgrid=True,
            gridcolor='rgba(0,0,0,0.1)'
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        plot_bgcolor='#FAFAFA',
        paper_bgcolor='white',
        hovermode="x unified"
    )

    # Add seasonal legend entries (as in first plot)
    fig.add_trace(go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        marker=dict(size=15, color=dry_season_color),
        name='Dry Season (Dec-May)'
    ))

    fig.add_trace(go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        marker=dict(size=15, color=wet_season_color),
        name='Wet Season (Jun-Nov)'
    ))

    fig.add_annotation(
        x=0.02,
        y=-0.15,
        xref="paper",
        yref="paper",
        align="left",
        text=f"<i><br>Gas cost calculated with {km_l} km/L<br>Electricity cost calculated with {km_kwh} km/kWh and {kwh_consumption} kWh consumption</i>",
        showarrow=False,
        font=dict(size=11, color="#666666")
    )

    # Show the plot
    fig.show()

In [322]:
plot_comparison_dataframe(40, 7, 200)

Statistical Test

In [323]:
def add_season_column(df):
    def assign_season(date):
        month = date.month
        if month == 12 or (1 <= month <= 5):  # November to May
            return "Dry"
        else:
            return "Wet"
    
    for date in df['Date']:
        df['Season'] = df['Date'].apply(assign_season)
        return df

def seasonality_test(_df: pd.DataFrame, column_to_check: str, time: str):
    res = []

    df = add_season_column(_df.copy())
    df = df[df['Date'].dt.year != 2025]
    
    if time == 'month':
        for i in df['Date'].dt.month.unique():
            time = df['Date'].dt.month
            res.append(df[time == i][column_to_check].values)
    elif time == 'year':
        for i in df['Date'].dt.year.unique():
            time = df['Date'].dt.year
            res.append(df[time == i][column_to_check].values)
    else:
        for i in df['Season'].unique():
            season = df['Season']
            res.append(df[season == i][column_to_check].values)

    
    return stats.kruskal(*res)

def print_all_kruskal(time: str):
    print(f"Time: {time}")
    r1 = seasonality_test(ron95_data, 'Average Price', time)
    print(f"{r1} for Gas, Avg Price")

    raw_df = pd.read_csv('WeeklyPumpPrices.csv', parse_dates=['Date'])
    test_data = raw_df[raw_df['Product'] == "Gasoline (RON95)"].sort_values('Date')
    test_data = test_data[(test_data['Date'] >= '2020-01-01') & 
                        (test_data['Date'] <= '2025-01-01')]
    test_data['Average Price'] = (test_data['Overall Range Min'] + test_data['Overall Range Max']) / 2
    rx = seasonality_test(test_data, 'Average Price', time)
    print(f"{rx} for Weekly Gas, Avg Price")

    r2 = seasonality_test(get_specific_kwh_df(300), 'Total Bill', time)
    print(f"{r2} for Elec, Total Bill w/ {300} kWh consumption")

    r3 = seasonality_test(make_comparison_dataframe(40, 7, 300), 'Difference', time)
    print(f"{r3} for Gas vs Elec cost/km, Difference w/ {40} km_l, {7} km_kWh, {300} kWh consumption")
    print()

print_all_kruskal('season')
print_all_kruskal('month')
print_all_kruskal('year')

Time: season
KruskalResult(statistic=0.42316939890710614, pvalue=0.5153597975802837) for Gas, Avg Price
KruskalResult(statistic=1.8164659289947023, pvalue=0.17773452341258905) for Weekly Gas, Avg Price
KruskalResult(statistic=0.005464480874309174, pvalue=0.9410723491398686) for Elec, Total Bill w/ 300 kWh consumption
KruskalResult(statistic=0.7868852459016296, pvalue=0.3750436235586161) for Gas vs Elec cost/km, Difference w/ 40 km_l, 7 km_kWh, 300 kWh consumption

Time: month
KruskalResult(statistic=1.7291803278688747, pvalue=0.9992440874031117) for Gas, Avg Price
KruskalResult(statistic=7.163614917238373, pvalue=0.785683027887575) for Weekly Gas, Avg Price
KruskalResult(statistic=1.1337704918032614, pvalue=0.9999049539174218) for Elec, Total Bill w/ 300 kWh consumption
KruskalResult(statistic=2.9409836065574098, pvalue=0.9914830281430439) for Gas vs Elec cost/km, Difference w/ 40 km_l, 7 km_kWh, 300 kWh consumption

Time: year
KruskalResult(statistic=45.49617486338795, pvalue=3.135160

Nutshell plot

In [324]:
def graph_nutshell_plot(km_l: float, km_kwh: float, kwh_consumption: int):
    ## 18, 7, 300, 40
    nutshell_df = make_comparison_dataframe(km_l, km_kwh, kwh_consumption)

    gas_color = '#dc267f'  # Red
    electricity_color = '#648fff'  # Blue
    vline_color = '#ffb000'
    text_color = '#FFFFFF'
    bg_color = '#1B181C'

    fig = go.Figure()

    fig.update_layout(
        font=dict(
            color=text_color,
            family="Roboto",
        )
    )

    line_width = 1

    ## vertical lines
    luzon_lockdown = datetime(2020, 3, 1)
    lockdown_reduced = datetime(2021, 10, 1)
    ukraine_invasion = datetime(2022, 2, 1)
    erc_order = datetime(2024, 6, 1)
    annotation_y = 1.75
    annotation_font = 10
    fig.update_layout(
        shapes=[
            dict(
                type="line",
                x0=luzon_lockdown,
                x1=luzon_lockdown,
                line=dict(color=vline_color, width=line_width, dash="dash")
            ),
            dict(
                type="line",
                x0=lockdown_reduced,
                x1=lockdown_reduced,
                line=dict(color=vline_color, width=line_width, dash="dash")
            ),
            dict(
                type="line",
                x0=ukraine_invasion,
                x1=ukraine_invasion,
                line=dict(color=vline_color, width=line_width, dash="dash")
            ),
            dict(
                type="line",
                x0=erc_order,
                x1=erc_order,
                line=dict(color=vline_color, width=line_width, dash="dash")
            ),
        ],
        annotations=[
            dict(
                x=luzon_lockdown,
                y=annotation_y,
                xref="x",
                showarrow=False,
                text="Luzon ECQ Start",
                textangle=-90,
                xanchor="right",
                yanchor="bottom",
                font=dict(color=vline_color, size=annotation_font),
            ),
            dict(
                x=lockdown_reduced,
                y=annotation_y,
                xref="x",
                showarrow=False,
                text="ECQ to MECQ",
                textangle=-90,
                xanchor="right",
                yanchor="bottom",
                font=dict(color=vline_color, size=annotation_font),
            ),
            dict(
                x=ukraine_invasion,
                y=annotation_y,
                xref="x",
                showarrow=False,
                text="Russia-Ukraine War",
                textangle=-90,
                xanchor="right",
                yanchor="bottom",
                font=dict(color=vline_color, size=annotation_font),
            ),
            dict(
                x=erc_order,
                y=annotation_y,
                xref="x",
                showarrow=False,
                text="ERC Order: Energy Costs",
                textangle=-90,
                xanchor="right",
                yanchor="bottom",
                font=dict(color=vline_color, size=annotation_font),
            )
        ],
    )

    # Add Gas line plot
    fig.add_trace(go.Scatter(
        x=nutshell_df['Date'],
        y=nutshell_df['Gas cost/km'],
        mode='lines+markers',
        line=dict(color=gas_color, width=2),
        marker=dict(size=8),
        name='HEV Gas Cost/km',
        hovertemplate="<b>%{x|%b %Y}</b><br>HEV Gas Cost: PHP %{y:.3f}<extra></extra>",
    ))

    fig.add_trace(go.Scatter(
        x=nutshell_df['Date'],
        y=nutshell_df['Electricity cost/km'],
        mode='lines+markers',
        line=dict(color=electricity_color, width=2),
        marker=dict(size=8),
        name='EV Electricity Cost/km',
        hovertemplate="<b>%{x|%b %Y}</b><br>EV Electricity Cost: PHP %{y:.3f}<extra></extra>",
    ))

    fig.update_layout(
        title={
            'text': "<b>Gas (HEV) VS Electricity (EV) Cost per Kilometer (2020-2025)</b>",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'font': dict(size=18)
        },
        xaxis=dict(
            title="Date",
            range=[nutshell_df['Date'].min(), nutshell_df['Date'].max()],
            tickformat="%Y",
            dtick="M12",  # Yearly ticks
            showgrid=False,
            showline=True,
            linecolor='white'
        ),
        yaxis=dict(
            title="Cost per km (PHP)",
            showgrid=False,
            showline=True,
            linecolor='white',
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        plot_bgcolor=bg_color,
        paper_bgcolor=bg_color,
        hovermode="x unified",
    )

    for date in nutshell_df['Date']:
        month = date.month
        if month != 1:
            continue
        gas_cost = nutshell_df.loc[nutshell_df['Date'] == date]['Gas cost/km']
        elec_cost = nutshell_df.loc[nutshell_df['Date'] == date]['Electricity cost/km']
        
        val = max(elec_cost.iloc[0], gas_cost.iloc[0])
        fig.add_shape(
            go.layout.Shape(
                type="line",
                x0=date, y0=0.2,  # Start at the x-axis (y=0)
                x1=date, y1=val,  # End at the data point
                xref="x", yref="y",
                line=dict(color="lightgray", width=1)
            )
        )

    fig.add_annotation(
        x=0.8,
        y=-0.25,
        xref="paper",
        yref="paper",
        align="left",
        text=f"""
        Gas cost calculated with {km_l} km/L<br>
        Electricity cost calculated with {km_kwh} km/kWh and {kwh_consumption} kWh household consumption<br>
        Sources: DoE NCR Prevailing Pump Prices, Meralco Rates
        """,
        showarrow=True,
        font=dict(size=11, color=text_color, family="Roboto Condensed")
    )

    # Show the plot
    fig.show()

graph_nutshell_plot(40, 7, 200)

In [325]:
make_comparison_dataframe(40, 7, 200)

,Date,Gas cost/km,Electricity cost/km,Difference
0,2020-01-01,1.28675,1.350329,-0.063579
1,2020-02-01,1.229125,1.266043,-0.036918
2,2020-03-01,1.1275,1.270014,-0.142514
3,2020-04-01,0.936375,1.285014,-0.348639
4,2020-05-01,1.008875,1.249543,-0.240668
5,2020-06-01,1.116,1.246457,-0.130457
6,2020-07-01,1.192125,1.242371,-0.050246
7,2020-08-01,1.20575,1.213014,-0.007264
8,2020-09-01,1.174625,1.204114,-0.029489
9,2020-10-01,1.166375,1.221429,-0.055054
